In [8]:
import numpy as np
import matplotlib.pyplot as pyplot
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.model_selection import train_test_split, learning_curve, StratifiedShuffleSplit

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

import DataPreprocessing

In [9]:
##Obtain & process data NPY
me_train, mom_train = DataPreprocessing.npy('NLO/MG_uux/me_1000000.npy', 'NLO/MG_uux/mom_1000000.npy', [], 1000, frac=0.1)
me_test, mom_test = DataPreprocessing.npy('NLO/MG_uux/me_300000.npy', 'NLO/MG_uux/mom_300000.npy', [], 1000, frac=0.1)

[[ 0.01255251  0.00058765 -0.00094296 -0.00062864]
 [ 0.0210088   0.00098354 -0.0015782  -0.00105214]
 [ 0.01483563  0.00069454 -0.00111447 -0.00074298]
 ...
 [ 0.01729205  0.00080954 -0.001299   -0.000866  ]
 [ 0.01509194  0.00070654 -0.00113372 -0.00075582]
 [ 0.0204844   0.00095899 -0.00153881 -0.00102587]]
[[ 0.01887437  0.00088361 -0.00141786 -0.00094524]
 [ 0.0160254   0.00075024 -0.00120385 -0.00080256]
 [ 0.01617337  0.00075717 -0.00121496 -0.00080997]
 ...
 [ 0.01997483  0.00093513 -0.00150053 -0.00100035]
 [ 0.01210853  0.00056687 -0.00090961 -0.0006064 ]
 [ 0.02163157  0.00101269 -0.00162499 -0.00108333]]


In [18]:
print(me_test[:,1]/me_test[:,0])

[0.0468156 0.0468156 0.0468156 ... 0.0468156 0.0468156 0.0468156]


In [13]:
def calc_RMSE(mom_train, mom_test, me_train, me_test, var_name):
    ##Data transformation
    pipeline = Pipeline([
        ('scaler', StandardScaler()), #Rescale Data.
        ('kbins', KBinsDiscretizer(n_bins=100)) #Checking convergence as bins.
    ])
 
    mom_train = pipeline.fit_transform(mom_train) #Rescale on training set
    mom_test = pipeline.transform(mom_test) #Rescale on test set

    
    linreg = LinearRegression().fit(mom_train, me_train) #Linear fit
    pred = linreg.predict(mom_test) #Prediction on test set
    
    mse = mean_squared_error(me_test, pred) #Mean squared error on test set
    print('{} RMSE: {}'.format(var_name, np.sqrt(mse)))

    perc = np.mean(100*np.divide(np.abs(me_test - pred), me_test))
    print('{} Percentage Error: {}'.format(var_name, perc))
    
for i, name in enumerate(['Born', 'Real', 'Sing', 'Doub']):
    calc_RMSE(mom_train, mom_test, me_train[:,i], me_test[:,i], name)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:193: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:193: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 4 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)


Born RMSE: 6.514233202993989e-05
Born Percentage Error: 0.3326042403650782


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:193: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:193: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 4 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)


Real RMSE: 3.049677385189429e-06
Real Percentage Error: 0.33260423956126833


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:193: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:193: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 4 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)


Sing RMSE: 4.893565790976995e-06
Sing Percentage Error: -0.3326042391826366


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:193: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:193: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 4 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)


Doub RMSE: 3.262377186895427e-06
Doub Percentage Error: -0.33260423934815153


Plotting Learning Curve

## Accuracy Vs Number of Bins